# 2.0 Building an AI-Powered Network Assistant

## **Introduction**

In this workshop, we will create an AI-powered assistant for managing Cisco IOS-XR devices using LangChain, GPT-4, and Netmiko. The assistant will assist network engineers in executing commands, analyzing logs, and troubleshooting network issues.

By the end of this workshop, you will:
- Understand how to automate network tasks using LangChain and GPT-4.
- Integrate SSH-based device interaction with Netmiko.
- Build an interactive CLI assistant for network management.
- Use the ReAct framework for structured network troubleshooting.

## **Workshop Outline**
1. Set up tools for command execution and log analysis.
2. Define a custom prompt for the assistant.
3. Integrate everything into a LangChain-powered agent.
4. Test the assistant in a real-world scenario.

🚀 Let’s get started and take your network management to the next level!

### **Complete the Following Pre-Requisites**  
1. Select the kernel: **"Python(ai-agent)"**  
2. Perform **Clear all outputs**

### Step 1: Warning Control

In this cell, we are importing the `warnings` module and suppressing any warning messages that may appear during the execution of the code.  

- **`import warnings`**: This imports the Python `warnings` module, which is used to handle warning messages that are generated during code execution.
- **`warnings.filterwarnings('ignore')`**: This command tells Python to ignore all warnings. It's commonly used in notebooks to keep the output clean, especially when you know that certain warnings are not critical to the execution or the purpose of the notebook.

This step is helpful for reducing unnecessary clutter in the output, ensuring that only the important results are displayed.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Step 2: Importing Libraries and Loading Environment Variables

In this step, we import essential libraries and load environment variables for the AI network assistant:

- **LangChain**:  
  - `Tool`, `initialize_agent`, `AgentType`: Set up the agent's behavior and interaction.
  - `ChatOpenAI`: Enables natural language interaction with OpenAI models.

- **Netmiko**:  
  - `ConnectHandler`: Establishes SSH connections with network devices.

- **Python's `re` module**: Used for analyzing logs with regular expressions.

- **Environment Variables**:  
  - `load_dotenv`: Loads sensitive information from a `.env` file to keep credentials secure.

This step sets up the libraries and environment for building the assistant.


In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI  # Use ChatGPT models if needed
from netmiko import ConnectHandler
import re

from dotenv import load_dotenv
_ = load_dotenv()

### Step 3: Defining Functions

In this cell, we define two key functions that help the assistant interact with the Cisco device and analyze logs:

#### **`execute_cisco_command(command: str)`**
This function connects to a Cisco device, runs a command, and returns the result.  
- It uses connection details (IP, username, password) to connect to the device via SSH.
- The command you specify is sent to the device, and the output is returned.
- If there’s an error while connecting or running the command, it returns an error message.

This function is used to get information or configurations from the device.

#### **`analyze_logs(log_data: str)`**
This function checks the logs for any errors or warnings.
- It looks for the words "error" or "warning" in the logs.
- If any are found, it returns a list of them. If nothing is found, it returns "No issues found in the logs."

This function helps troubleshoot by identifying issues in the logs.


In [ ]:
def execute_cisco_command(command: str):
    device = {
        "device_type": "cisco_ios",
        "host": "198.18.128.3",  # Replace with your device IP
        "username": "cisco",  # Replace with your username
        "password": "cisco123",  # Replace with your password
    }
    try:
        connection = ConnectHandler(**device)
        output = connection.send_command(command)
        connection.disconnect()
        return output
    except Exception as e:
        return f"Error: {e}"
    
def analyze_logs(log_data: str):
    issues = []
    if "error" in log_data.lower():
        errors = re.findall(r"error.*", log_data, re.IGNORECASE)
        issues.extend(errors)
    if "warning" in log_data.lower():
        warnings = re.findall(r"warning.*", log_data, re.IGNORECASE)
        issues.extend(warnings)
    return "\n".join(issues) if issues else "No issues found in the logs."


### Step 4: Defining the Custom Prompt

In this step, we define a **custom prompt** that guides the AI assistant in responding to network-related queries. The prompt follows a structured process to ensure clear and actionable responses.

#### **Key Elements of the Prompt:**
- **Smart Network Assistant**: The assistant helps troubleshoot and manage network devices, check configurations, analyze logs, and provide insights.
- **Process Flow**: The assistant follows a series of steps for each task:
  1. **Thought**: Evaluate the problem or query.
  2. **Action**: Choose the appropriate tool and input.
  3. **PAUSE**: Wait for the action result.
  4. **Observation**: Review the result.
  5. **Answer**: Provide a response based on the observations.

#### **Available Tools**:
- **Execute Cisco Command**: Runs commands on Cisco devices to retrieve configurations or status information.
- **Analyze Logs**: Parses logs to detect errors or warnings and suggests solutions.

#### **Example Interaction**:
- **Query**: Check the current interface statuses on the router.
- **Thought**: The assistant decides to run the `show ip interface brief` command.
- **Action**: Execute the command.
- **Observation**: The assistant reviews the interface statuses.
- **Answer**: Provides the status and identifies any issues.

This structured prompt helps the assistant respond in a clear, logical, and professional manner.

In [ ]:
from langchain.prompts import PromptTemplate

# Define the custom prompt
prompt_template = """
You are a Smart Network Assistant designed to help network engineers troubleshoot and manage network devices efficiently. 
Your primary tasks include checking device configurations, analyzing logs, and providing actionable insights for troubleshooting.

You operate in a loop of Thought, Action, PAUSE, Observation, and Answer:

1. Thought: Describe your reasoning for the problem or query.
2. Action: Choose one of the tools available to you, provide the required input, and return PAUSE.
3. Observation: Wait for the result of the action to proceed.
4. Answer: Provide the final output or solution based on your observations.

### Tools Available:
- Execute Cisco Command:
  - Example: Execute Cisco Command: show running-config
  - Use this tool to retrieve configurations or status from a device.

- Analyze Logs:
  - Example: Analyze Logs: [log content here]
  - Use this tool to parse logs, identify errors, warnings, or patterns, and suggest resolutions.

### Guidelines:
- Always prioritize clarity and precision in your responses.
- If a command or log needs clarification, politely ask for more details.
- Assume you are working in a professional network environment and maintain a concise, professional tone.

Example Interaction:

Query: Check the current interface statuses on the router.
Thought: I should retrieve the interface statuses using the show ip interface brief command.
Action: Execute Cisco Command: show ip interface brief
PAUSE

Observation: 
Interface              IP-Address      OK? Method Status                Protocol
FastEthernet0/0        192.168.1.1     YES manual up                    up
FastEthernet0/1        unassigned      YES unset  administratively down down

Answer: 
Here are the current interface statuses:
- FastEthernet0/0: IP 192.168.1.1 is UP and running.
- FastEthernet0/1: No IP assigned and the interface is DOWN.

Stay within your role as a Smart Network Assistant, and always aim to assist the user with their tasks.
""".strip()

custom_prompt = PromptTemplate(template=prompt_template, input_variables=[])


### Step 5: Defining Tools and Initializing the Agent

In this step, we define the tools and set up the AI agent that will interact with network devices and analyze logs.

#### **Defining the Tools**:
- **`cisco_tool`**: This tool enables the assistant to execute Cisco commands (e.g., `show running-config`) on devices and retrieve the output.
- **`log_tool`**: This tool analyzes logs for errors or warnings and provides insights based on the analysis.

#### **Setting up the Language Model**:
- **`llm` (Language Model)**: We use the `ChatOpenAI` model (GPT-4) for generating responses to user queries based on device data or logs. The `temperature=0` setting ensures more consistent and predictable responses.

#### **Initializing the Agent**:
- **`initialize_agent()`**: This function sets up the agent with the tools and language model. The agent will follow the custom prompt to make decisions based on available tools.
  - **Agent Type**: `ZERO_SHOT_REACT_DESCRIPTION` enables the agent to reason and act without pre-training specific knowledge.
  - **`verbose=True`**: Prints detailed logs of the agent’s decision-making process for better transparency.

By defining these tools and setting up the agent, we prepare the assistant to handle various network management tasks efficiently.


In [ ]:
# Define Tools
cisco_tool = Tool(
    name="Execute Cisco Command",
    func=execute_cisco_command,
    description="Executes a Cisco show command and retrieves its output."
)

log_tool = Tool(
    name="Analyze Logs",
    func=analyze_logs,
    description="Analyzes logs and reports any errors or warnings found."
)

llm = ChatOpenAI(temperature=0, model="gpt-4")  # Adjust model based on your requirements
tools = [cisco_tool, log_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"custom_prompt": custom_prompt}
)

### Step 6: Running the AI-Powered Network Assistant

In this final cell, we run the AI-powered assistant in an interactive loop, where the user can input commands or queries, and the assistant will respond accordingly.

#### **How It Works**:
- **`print("Welcome to the AI-Powered Network Assistant!")`**: This prints a welcome message when the program starts.
- **`while True:`**: This creates an infinite loop, allowing the user to enter commands or queries repeatedly.
  - **User Input**: The assistant waits for the user to type a command or query.
  - **`if user_input.lower() in ["exit", "quit"]:`**: If the user types "exit" or "quit", the loop ends, and the program prints "Goodbye!".
  - **`response = agent.run(user_input)`**: The assistant processes the user input using the previously defined agent and responds accordingly.
  - **`print(f"Response:\n{response}")`**: The response from the assistant is displayed to the user.
  - **Error Handling**: If there’s an issue while running the assistant (e.g., invalid command), it will print an error message.

This cell makes the assistant interactive, enabling real-time communication with the user to troubleshoot network issues or gather information from devices.


In [ ]:
if __name__ == "__main__":
    print("Welcome to the Smart Network Assistant!")
    while True:
        user_input = input("Enter your command or query: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        try:
            response = agent.run(user_input)
            print(f"Response:\n{response}")
        except Exception as e:
            print(f"Error: {e}")


### Step 7: Now let's run a UI for our AI Network Assistant.

Go to the terminal and lets run this example in the browser for you to interact with it seemlessly.

1. open a new Terminal and then choose command prompt(cmd)
2. run this command >streamlit run C:\Users\admin\Desktop\devwks\DEVWKS-2382\UI\network-assistant-ai-agent-UI-v2.0.py

## **Conclusion**

🎉 **Congratulations on Building Your AI-Powered Network Assistant!** 🎉

You've successfully created an assistant that:

✅ **Executes Cisco commands** over SSH and retrieves results.

✅ **Analyzes logs** to detect errors and provide insights.

✅ **Interprets user queries** and offers actionable advice.

This assistant showcases the power of AI and network automation, enhancing efficiency and reducing troubleshooting time.

### **Next Steps**
1. **Enhance Features**  
   - Add more diagnostic tools.  
   - Expand the prompt to support more complex queries.

2. **Secure Deployment**  
   - Integrate secure credential storage.  
   - Deploy in a controlled production environment.

3. **Experiment and Scale**  
   - Explore use cases like **configuration management** and **real-time monitoring**.  
   - Scale for **multi-device environments**.